In [1]:
import numpy as np
from six.moves import cPickle as pickle
from scipy import ndimage
from scipy.misc import imresize, imsave

pixels_width = 32
pixels_height = 32
file_end = ".png"
pixels_depth = 255.0

In [2]:
pickleFile = open('svhn.pickle', 'r')
data = pickle.load(pickleFile)
training = data['training']
testing = data['testing']
valid = data['valid']

In [3]:
def removeZeros(arr):
    newarr = []
    for index in range(len(arr)):
        if arr[index] > 0:
            newarr.append(arr[index])
    return newarr

def getLeftBound(lefts):
    return int(min(lefts))

def getTopBound(tops):
    return int(min(tops))

def getBottomBound(top, heights):
    return int(max(heights) + top)

def getRightBound(lefts, widths):
    return int(max(lefts) + max(widths))

In [4]:
def manipulateImage(path, dataset, index):
    name = dataset['images'][index]
    imageFile = path + name + file_end
    imageData = np.average(ndimage.imread(imageFile),axis=2)
    lefts = removeZeros(dataset['lefts'][index])
    widths = removeZeros(dataset['widths'][index])
    tops = removeZeros(dataset['tops'][index])
    heights = removeZeros(dataset['heights'][index])
    
    left = getLeftBound(lefts)
    if left < 0: left = 0
    right = getRightBound(lefts, widths)
    top = getTopBound(tops)
    if top < 0: top = 0
    bottom = getBottomBound(top, heights)
    
    cutImage = imageData[top:bottom, left:right]
    if cutImage.size == 0:
        cutImage = imageData
    resizedData = imresize(cutImage, (pixels_height, pixels_width))
    resizedData = (resizedData.astype(float) - pixels_depth / 2) / pixels_depth
    return resizedData

def generateNewImages(path, dataset):
    indices = dataset['images'].shape[0]
    generatedData = np.ndarray(shape=(indices, pixels_height, pixels_width), dtype=np.float32)
    counter = 0
    for index in range(indices):
        try:
            image = manipulateImage(path, dataset, index)
            generatedData[index, :, :] = image
        except Exception, e:
            #print(index)
            #print(dataset['images'][index])
            counter = counter +1
    print(counter)
    return generatedData

In [5]:
training_X = generateNewImages("train/", training)
testing_X = generateNewImages("test/", testing)
valid_X = generateNewImages("extra/", valid)

22
11
9


In [6]:
print(training_X.shape)
print(testing_X.shape)
print(valid_X.shape)

(33402, 32, 32)
(13068, 32, 32)
(10000, 32, 32)


In [7]:
training_Y = training['labels']
testing_Y = testing['labels']
valid_Y = valid['labels']

In [8]:
training_Y[0]

array([  1.,   9.,  10.,  10.,  10.,  10.])

In [9]:
def createDataLabelDict(X,Y):
    return {
        'data': X,
        'label': Y
    }

tensorflow_file = 'tensorflow_data.pickle'
try:
    with open(tensorflow_file, 'wb') as f:
        dump = {
            'train': createDataLabelDict(training_X, training_Y),
            'test': createDataLabelDict(testing_X, testing_Y),
            'valid': createDataLabelDict(valid_X, valid_Y)
        }
        pickle.dump(dump, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', tensorflow_file, ':', e)